In [56]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

In [ ]:
# import labels for the shapes
shape_labels = np.genfromtxt('../FiltrationsForGNNs/Gudhi Shape Dataset/shape_labels.csv', delimiter=',', skip_header=1)
shape_labels = shape_labels.astype(int)[:,2]
print(shape_labels)

[1 1 1 ... 0 0 0]


2000

In [4]:
num_samples = 200 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(shape_labels), size=num_samples, replace=False)
base = '../FiltrationsForGNNs/Gudhi Shape Dataset/'
# Select the corresponding data and labels
laplacians = []
vr_persistence_images = []
abstract_persistence_images = []
selected_labels = []

for i in random_indices:
    laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    selected_labels.append(shape_labels[i])

# Convert selected labels to NumPy array
selected_labels = np.array(selected_labels)

# Print a summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(abstract_persistence_images).shape}")
print(f"Shape of selected labels: {selected_labels.shape}")

Randomly selected 200 samples.
Shape of laplacians: (200, 1000, 1000)
Shape of VR persistence images: (200, 100, 100)
Shape of abstract persistence images: (200, 100, 100)
Shape of selected labels: (200,)


In [62]:
selected_labels

array([1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0])

In [5]:
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data]
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [6]:
class CNN(nn.Module):
    def __init__(self, input_shape, num_classes=2):
        super(CNN, self).__init__()
        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Adaptive Pooling to resize to 100x100
        self.adaptive_pool = nn.AdaptiveAvgPool2d((100, 100))
        
        # Dynamically calculate input size to fc1
        self.feature_size = self._get_feature_size(input_shape)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(self.feature_size, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_feature_size(self, input_shape):
        # Create a dummy input to calculate size after conv and pooling
        dummy_input = torch.zeros(1, 1, *input_shape)
        x = self.pool(F.relu(self.conv1(dummy_input)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to get 100x100 size
        x = self.adaptive_pool(x)
        return x.numel()  # Number of elements after flattening

    def forward(self, x):
        # Apply convolutional layers with pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        # Apply adaptive pooling to resize to 100x100
        x = self.adaptive_pool(x)
        
        # Flatten and pass through fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [7]:
class DualInputCNN(nn.Module):
    def __init__(self, input_shape1, input_shape2, num_classes=2):
        super(DualInputCNN, self).__init__()

        # Laplacian input path with additional pooling to reduce to 100x100
        self.conv1_lap = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_lap = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool_lap = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions
        self.adaptive_pool_lap = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Persistence image input path (no pooling)
        self.conv1_pers = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_pers = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.adaptive_pool_pers = nn.AdaptiveAvgPool2d((100, 100))  # Resize to 100x100
        
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 100 * 100 + 32 * 100 * 100, 128)  # Adjusted for 100x100 input
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x1, x2):
        # Laplacians path (downsampling to 100x100)
        x1 = F.relu(self.conv1_lap(x1))
        x1 = self.pool_lap(x1)  # First pool: 250x250 -> 125x125
        x1 = F.relu(self.conv2_lap(x1))
        x1 = self.pool_lap(x1)  # Second pool: 125x125 -> 62x62
        x1 = self.adaptive_pool_lap(x1)  # Resize to 100x100
        
        # Persistence images path (no pooling)
        x2 = F.relu(self.conv1_pers(x2))
        x2 = F.relu(self.conv2_pers(x2))
        x2 = self.adaptive_pool_pers(x2)  # Ensure persistence images are 100x100
        
        # Concatenate along dim=1 (channels)
        x = torch.cat((x1, x2), dim=1)  # Concatenates the outputs along the channel axis

        # Flatten for fully connected layer
        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cpu


In [54]:
def train_single_input(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass with single input
        output = model(data)  # Forward through the single-input model
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update loss and accuracy
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Update progress bar
        tk0.set_postfix(loss=loss.item())

    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def test_single_input(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)  # Forward through the single-input model
            loss = criterion(output, target)
            
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    
    # Calculate additional metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return avg_loss, accuracy, precision, recall, f1


In [10]:
def train_and_test_single_input(data_type, data, labels, input_shape):
    # Create dataset and split into train/test sets
    dataset = ShapeDataset(data, labels)
    train_data, test_data, train_labels, test_labels = train_test_split(
        dataset.data, dataset.labels, test_size=0.2, random_state=42
    )

    # Convert to custom Dataset format for train and test sets
    train_dataset = torch.utils.data.TensorDataset(torch.stack(train_data), train_labels)
    test_dataset = torch.utils.data.TensorDataset(torch.stack(test_data), test_labels)

    # Create DataLoaders
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Define CNN model with input_shape
    model = CNN(input_shape=input_shape, num_classes=len(set(labels))).to(device)

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()  # For multi-class classification

    epoch_results = []

    # Training loop
    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        print(f"Training model for {data_type} - Epoch {epoch}/{num_epochs}")
        train_loss, accuracy = train_single_input(model, device, train_dataloader, optimizer, criterion, epoch)
        test_loss, accuracy, precision, recall, f1 = test_single_input(model, device, test_dataloader, criterion)

        # Store results
        epoch_results.append({
            'Epoch': epoch,
            'Test Loss': test_loss,
            'Test Accuracy (%)': accuracy,
            'Test Precision (%)': precision * 100,
            'Test Recall (%)': recall * 100,
            'Test F1 Score': f1
        })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    print(f"\nTesting results for {data_type}:")
    print(epoch_results_df)
    return model


In [ ]:
def train_and_test_dual_input(data_type, data1, data2, labels, input_shape1, input_shape2):
    dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data1]  # Shape [1, 100, 100]
    dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data2]  # Shape [1, 100, 100]
    labels = torch.tensor(labels, dtype=torch.long)

    train_data1, test_data1, train_data2, test_data2, train_labels, test_labels = train_test_split(
        dataset1, dataset2, labels, test_size=0.2, random_state=42
    )

    train_dataset = torch.utils.data.TensorDataset(
        torch.stack(train_data1), torch.stack(train_data2), train_labels
    )
    test_dataset = torch.utils.data.TensorDataset(
        torch.stack(test_data1), torch.stack(test_data2), test_labels
    )

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = DualInputCNN(input_shape1=input_shape1, input_shape2=input_shape2, num_classes=len(set(labels))).to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    epoch_results = []

    num_epochs = 10
    for epoch in range(1, num_epochs + 1):
        print(f"Training model for {data_type} - Epoch {epoch}/5")
        
        # Training step
        train_loss, train_accuracy = train_dual_input(model, device, train_dataloader, optimizer, criterion, epoch)
        
        # Testing step
        test_loss, test_accuracy, precision, recall, f1 = test_dual_input(model, device, test_dataloader, criterion)
        
        # Storing results
        epoch_results.append({
            'Epoch': epoch,
            'Test Loss': test_loss,
            'Test Accuracy (%)': test_accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    print(f"\nTesting results for {data_type}:")
    print(epoch_results_df)
    return model


In [55]:
def train_dual_input(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    tk0 = tqdm(train_loader, total=len(train_loader))
    for batch_idx, (data1, data2, target) in enumerate(tk0):
        data1, data2, target = data1.to(device), data2.to(device), target.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass with dual input
        output = model(data1, data2)  # Forward through the dual-input model
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update loss and accuracy
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Update progress bar
        tk0.set_postfix(loss=loss.item())

    avg_loss = train_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy


def test_dual_input(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for data1, data2, target in test_loader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = model(data1, data2)  # Forward through the dual-input model
            loss = criterion(output, target)
            
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            # Collect all labels and predictions for additional metrics
            all_labels.extend(target.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    
    # Calculate additional metrics
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Return test loss, accuracy, and additional metrics
    return avg_loss, accuracy, precision, recall, f1


In [13]:
# Train and test for Laplacians (1000x1000 input)
lap = train_and_test_single_input("Laplacians", laplacians, selected_labels, input_shape=(1000, 1000))

Training model for Laplacians - Epoch 1/10


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 2/10


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 3/10


...

Training model for Laplacians - Epoch 9/10


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians - Epoch 10/10


  0%|          | 0/10 [00:00<?, ?it/s]


Testing results for Laplacians:
   Epoch  Test Loss  Test Accuracy (%)  Test Precision (%)  Test Recall (%)  \
0      1   5.966245               70.0           80.588235             70.0   
1      2   0.562329               45.0           20.250000             45.0   
2      3   0.198150               92.5           93.400000             92.5   
3      4   0.191942               95.0           95.416667             95.0   
4      5   0.154823               95.0           95.416667             95.0   
5      6   0.194205               95.0           95.416667             95.0   
6      7   0.131854               95.0           95.416667             95.0   
7      8   0.169439               95.0           95.416667             95.0   
8      9   0.139449               95.0           95.416667             95.0   
9     10   0.115736               95.0           95.416667             95.0   

   Test F1 Score  
0       0.657143  
1       0.279310  
2       0.923985  
3       0.949616  
4 

In [14]:
dual_vr = train_and_test_dual_input("Laplacians + VR Persistence Images", laplacians, vr_persistence_images, selected_labels, (1000, 1000), (100, 100))

Training model for Laplacians + VR Persistence Images - Epoch 1/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + VR Persistence Images - Epoch 2/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + VR Persistence Images - Epoch 3/5


...

Training model for Laplacians + VR Persistence Images - Epoch 9/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + VR Persistence Images - Epoch 10/5


  0%|          | 0/10 [00:00<?, ?it/s]


Testing results for Laplacians + VR Persistence Images:
   Epoch  Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0      1  36.266187               85.0   0.862088   0.850  0.846875
1      2   3.339916               62.5   0.795455   0.625  0.583164
2      3   0.427565               92.5   0.935714   0.925  0.925141
3      4   1.042344               92.5   0.935714   0.925  0.925141
4      5   0.252098               87.5   0.898148   0.875  0.871297
5      6   0.848984               92.5   0.935714   0.925  0.925141
6      7   0.003119              100.0   1.000000   1.000  1.000000
7      8   0.148613               95.0   0.955000   0.950  0.950125
8      9   0.001413              100.0   1.000000   1.000  1.000000
9     10   0.002278              100.0   1.000000   1.000  1.000000


In [15]:
dual_AP = train_and_test_dual_input("Laplacians + Abstract Persistence Images", laplacians, abstract_persistence_images, selected_labels, (1000, 1000), (100, 100))

Training model for Laplacians + Abstract Persistence Images - Epoch 1/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 2/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 3/5


...

Training model for Laplacians + Abstract Persistence Images - Epoch 9/5


  0%|          | 0/10 [00:00<?, ?it/s]

Training model for Laplacians + Abstract Persistence Images - Epoch 10/5


  0%|          | 0/10 [00:00<?, ?it/s]


Testing results for Laplacians + Abstract Persistence Images:
   Epoch  Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0      1   2.930052               82.5   0.826692   0.825  0.825330
1      2   0.542180               87.5   0.885338   0.875  0.875235
2      3   0.334232               92.5   0.926441   0.925  0.925141
3      4   0.189809               95.0   0.955000   0.950  0.950125
4      5   0.009176              100.0   1.000000   1.000  1.000000
5      6   0.013109               97.5   0.976316   0.975  0.975047
6      7   0.049952               97.5   0.976316   0.975  0.975047
7      8   0.270962               92.5   0.934000   0.925  0.923985
8      9   0.055290               97.5   0.976087   0.975  0.974921
9     10   0.060581               97.5   0.976316   0.975  0.975047


In [19]:
# import labels for the shapes
medical_shape_labels = np.genfromtxt('../FiltrationsForGNNs/Medical Dataset/shape_labels.csv', delimiter=',', skip_header=1)
medical_shape_labels = medical_shape_labels.astype(int)[:,2]
print(medical_shape_labels)
print(len(medical_shape_labels))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
162


In [22]:
num_samples = 50 # currently set to full dataset

# Generate random indices
random_indices = np.random.choice(len(medical_shape_labels), size=num_samples, replace=False)
base = '../FiltrationsForGNNs/Medical Dataset/'
# Select the corresponding data and labels
medical_laplacians = []
medical_vr_persistence_images = []
medical_abstract_persistence_images = []
medical_selected_labels = []

for i in random_indices:
    medical_laplacians.append(np.genfromtxt(f'{base}/shape_{i}_laplacian.csv', delimiter=',', skip_header=0))
    medical_vr_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_vr_persistence_image.csv', delimiter=',', skip_header=0))
    medical_abstract_persistence_images.append(np.genfromtxt(f'{base}/shape_{i}_abstract_persistence_image.csv', delimiter=',', skip_header=0))
    medical_selected_labels.append(medical_shape_labels[i])

# Convert selected labels to NumPy array
medical_selected_labels = np.array(medical_selected_labels)

# Print a summary
print(f"Randomly selected {num_samples} samples.")
print(f"Shape of laplacians: {np.array(medical_laplacians).shape}")
print(f"Shape of VR persistence images: {np.array(medical_vr_persistence_images).shape}")
print(f"Shape of abstract persistence images: {np.array(medical_abstract_persistence_images).shape}")
print(f"Shape of selected labels: {medical_selected_labels.shape}")

Randomly selected 50 samples.
Shape of laplacians: (50, 1000, 1000)
Shape of VR persistence images: (50, 100, 100)
Shape of abstract persistence images: (50, 100, 100)
Shape of selected labels: (50,)


In [63]:
medical_selected_labels

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1])

In [40]:
data_lap = ShapeDataset(medical_laplacians, medical_selected_labels)
data_vr = ShapeDataset(medical_vr_persistence_images, medical_selected_labels)
data_pi = ShapeDataset(medical_abstract_persistence_images, medical_selected_labels)

In [91]:
def validation_single(model, data):
    test_dataloader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=False)

    epoch_results = []
    # Testing step
    test_loss, test_accuracy, precision, recall, f1 = test_single_input(model, device, test_dataloader, nn.CrossEntropyLoss())


    # Storing results
    epoch_results.append({
    'Test Loss': test_loss,
    'Test Accuracy (%)': test_accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
    })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    y = []
    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = lap(data)  # Forward through the single-input model
            _, predicted = output.max(1)
            y.append(predicted)
    return epoch_results_df, y

In [92]:
results,output = validation_single(lap, data_lap)

c:\Users\alanp\anaconda3\envs\FiltrationsForGNNs\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def validation_dual(model, data1, data2, label):
    dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data1]  # Shape [1, 100, 100]
    dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in data2]  # Shape [1, 100, 100]
    labels = torch.tensor(label, dtype=torch.long)
    test_dataset = torch.utils.data.TensorDataset(
        torch.stack(dataset1), torch.stack(dataset2), labels
    )
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_loss, test_accuracy, precision, recall, f1 = test_dual_input(dual_vr, device, test_dataloader, nn.CrossEntropyLoss())
    epoch_results = []

    # Storing results
    epoch_results.append({
        'Test Loss': test_loss,
        'Test Accuracy (%)': test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

    # Convert results to DataFrame for tabular output
    epoch_results_df = pd.DataFrame(epoch_results)
    y = []
    with torch.no_grad():
        for data1, data2, target in test_dataloader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = dual_vr(data1, data2)  # Forward through the dual-input model
            _, predicted = output.max(1)
            y.append(predicted)
    return epoch_results_df, y

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [72]:
dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in medical_laplacians]  # Shape [1, 100, 100]
dataset1[0].shape

torch.Size([1, 1000, 1000])

In [81]:
dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in medical_laplacians]  # Shape [1, 100, 100]
dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in medical_vr_persistence_images]  # Shape [1, 100, 100]
labels = torch.tensor(medical_selected_labels, dtype=torch.long)

test_dataset = torch.utils.data.TensorDataset(
    torch.stack(dataset1), torch.stack(dataset2), labels
    )


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

test_loss, test_accuracy, precision, recall, f1 = test_dual_input(dual_vr, device, test_dataloader, nn.CrossEntropyLoss())
epoch_results = []

# Storing results
epoch_results.append({
'Test Loss': test_loss,
'Test Accuracy (%)': test_accuracy,
'Precision': precision,
'Recall': recall,
'F1 Score': f1
})

# Convert results to DataFrame for tabular output
epoch_results_df = pd.DataFrame(epoch_results)
print(epoch_results_df)

   Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0   4.287778               32.0   0.661333    0.32  0.248539


In [86]:
y = []
with torch.no_grad():
        for data1, data2, target in test_dataloader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = dual_vr(data1, data2)  # Forward through the dual-input model
            _, predicted = output.max(1)
            y.append(predicted)

y

[tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
         0, 1, 1, 1, 1, 1, 0, 1]),
 tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]

In [87]:
dataset1 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in medical_laplacians]  # Shape [1, 100, 100]
dataset2 = [torch.tensor(d, dtype=torch.float32).unsqueeze(0) for d in medical_abstract_persistence_images]  # Shape [1, 100, 100]
labels = torch.tensor(medical_selected_labels, dtype=torch.long)

test_dataset = torch.utils.data.TensorDataset(
    torch.stack(dataset1), torch.stack(dataset2), labels
    )


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

test_loss, test_accuracy, precision, recall, f1 = test_dual_input(dual_AP, device, test_dataloader, nn.CrossEntropyLoss())
epoch_results = []

# Storing results
epoch_results.append({
'Test Loss': test_loss,
'Test Accuracy (%)': test_accuracy,
'Precision': precision,
'Recall': recall,
'F1 Score': f1
})

# Convert results to DataFrame for tabular output
epoch_results_df = pd.DataFrame(epoch_results)
print(epoch_results_df)

   Test Loss  Test Accuracy (%)  Precision  Recall  F1 Score
0  19.548151               74.0     0.5476    0.74  0.629425


c:\Users\alanp\anaconda3\envs\FiltrationsForGNNs\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [88]:
y = []
with torch.no_grad():
        for data1, data2, target in test_dataloader:
            data1, data2, target = data1.to(device), data2.to(device), target.to(device)
            output = dual_AP(data1, data2)  # Forward through the dual-input model
            _, predicted = output.max(1)
            y.append(predicted)

y

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]